In [1]:
import sys

sys.path.append("..")
from omegaconf import OmegaConf
from sgm.util import instantiate_from_config

config = OmegaConf.load(
    "/vol/paramonos2/projects/antoni/code/Personal/generative-models/configs/example_training/svd_training.yaml"
)
print(config)
config["model"]["params"][
    "ckpt_path"
] = "/vol/paramonos2/projects/antoni/code/Personal/generative-models/checkpoints/svd.safetensors"
config["model"]["params"]["network_config"]["params"]["fine_tuning_method"] = None
model = instantiate_from_config(config.model)

/vol/paramonos2/projects/antoni/miniconda3/envs/svd/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'model': {'base_learning_rate': 3e-05, 'target': 'sgm.models.diffusion.DiffusionEngine', 'params': {'input_key': 'latents', 'no_log_keys': ['audio_emb', 'fps_id', 'motion_bucket_id', 'cond_aug'], 'scale_factor': 0.18215, 'disable_first_stage_autocast': True, 'ckpt_path': 'checkpoints/svd.safetensors', 'use_lora': True, 'lora_config': {'target_replace_module': ['SpatialVideoTransformer'], 'r': 16, 'loras': None, 'verbose': False, 'dropout_p': 0.0, 'scale': 1.0}, 'denoiser_config': {'target': 'sgm.modules.diffusionmodules.denoiser.Denoiser', 'params': {'scaling_config': {'target': 'sgm.modules.diffusionmodules.denoiser_scaling.VScalingWithEDMcNoise'}}}, 'network_config': {'target': 'sgm.modules.diffusionmodules.video_model.VideoUNet', 'params': {'adm_in_channels': 768, 'num_classes': 'sequential', 'use_checkpoint': True, 'in_channels': 8, 'out_channels': 4, 'model_channels': 320, 'attention_resolutions': [4, 2, 1], 'num_res_blocks': 2, 'channel_mult': [1, 2, 4, 4], 'num_head_channels': 

In [ ]:
from safetensors.torch import load_file

model_ckpt = load_file("../checkpoints/svd.safetensors")

print(model_ckpt.keys())

In [2]:
unet = model.model.diffusion_model

In [ ]:
from sgm.modules.diffusionmodules.adapters.lora import apply_lora
import torch

# activate_lora = add_lora_to(unet, ["SpatialVideoTransformer"], search_class=[torch.nn.Linear], r=32, dropout=0, lora_bias="none")
apply_lora(unet, filters=[".input_blocks"], rank=16, all_modules_in_filter=True)

In [ ]:
from sgm.modules.diffusionmodules.adapters.lora import LoRA

lora = {}
lora["weights"] = torch.nn.ModuleList()
for module in unet.modules():
    if isinstance(module, LoRA) or (
        hasattr(module, "_fsdp_wrapped_module") and isinstance(module._fsdp_wrapped_module, LoRA)
    ):
        print(module.requires_grad_())
        lora["weights"].append(module)

In [ ]:
lora

In [ ]:
for param in unet.parameters():
    if param.requires_grad:
        print("requires grad")
    else:
        print("does not require grad")

In [ ]:
for p in unet.input_blocks[0].parameters():
    print(p)

In [ ]:
for name, module in unet.named_modules():
    print(name)

In [4]:
from sgm.modules.diffusionmodules.adapters.lora import get_module_names

module_names = get_module_names(unet, filters=[".transformer_blocks"], all_modules_in_filter=True)

In [5]:
from peft import LoraModel, LoraConfig

lora_config = LoraConfig(
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=module_names,
)
lora_unet = LoraModel(unet, lora_config, "bite")

In [14]:
lora_unet.input_blocks[4].disable_adapter()

AttributeError: 'TimestepEmbedSequential' object has no attribute 'disable_adapter'